### Tecnolgías del Lenguaje. Entregable 4
---
#### Reducción basada en similitud

Se aplica un **umbral de similitud** para asegurar que solo se conserven aquellos posts que realmente están relacionados con el rasgo psicológico que se está analizando. Las similitudes bajas o negativas indican que el contenido del post no guarda relación significativa con el vector representante del rasgo, por lo que incluirlos introduciría ruido y debilitaría la interpretación. Al mantener únicamente los posts con una similitud superior a 0.3, se garantiza que el conjunto final esté compuesto por ejemplos más coherentes, relevantes y explicativos, lo que permite obtener conclusiones más fiables sobre cómo se manifiesta cada rasgo en el lenguaje de los usuarios.

El resultado es un archivo reducido e informativo, almacenado en `material/top50_posts_per_user_reduced.csv`

In [1]:
import pandas as pd

# Cargar el archivo original
df = pd.read_csv("material/top50_posts_per_user.csv")

# Asumiendo que la columna se llama 'similarity' o 'sim'
# Ajusta el nombre si es diferente
df_filtrado = df[df["similarity"] > 0.3]

# Guardar el archivo reducido
df_filtrado.to_csv("material/top50_posts_per_user_reduced.csv", index=False)

print("Filtrado completado. Posts restantes:", len(df_filtrado))

Filtrado completado. Posts restantes: 192896
